In [ ]:
%pip install transformers
%pip install datasets
%pip install PIL
%pip install numpy

In [ ]:
import tensorflow as tf
from datasets import load_dataset

DATASET_SIZE = 23298
dataset = load_dataset("microsoft/cats_vs_dogs")

def to_rgb(example):
  if example["image"].mode != "RGB":
    example["image"] = example["image"].convert("RGB")
  return example

dataset = load_dataset("microsoft/cats_vs_dogs")
dataset = dataset.map(to_rgb)  # now every image is RGB

tf_dataset = dataset["train"].to_tf_dataset(
  shuffle=True,
  batch_size=None
)

def dict_to_tuple(data):
    return tf.cast(data['image'], tf.float32) / 255.0, tf.cast(data['labels'], tf.float32)

def resize(image, label):
    image = tf.image.resize(image, [160, 90])
    return image, label

tf_dataset = tf_dataset.map(dict_to_tuple).map(resize)

In [ ]:
val_size = 1000
test_size = 500
train_size = DATASET_SIZE - val_size - test_size

train_db = tf_dataset.take(train_size)
valid_db = tf_dataset.skip(train_size).take(val_size)
test_db = tf_dataset.skip(train_size + val_size).take(test_size)

batch_size = 64
train_db = train_db.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_db = test_db.batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid_db = valid_db.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
from functools import partial

tf.random.set_seed(42)
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")

model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[160, 90, 3]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_db, validation_data=valid_db, epochs=10)

In [ ]:
model.evaluate(test_db)

In [ ]:
import tensorflow as tf

new_model = tf.keras.models.load_model('c-d-clas.mk1.keras')

# Check its architecture
new_model.summary()

In [ ]:
from PIL import Image
import numpy as np
im = Image.open('3.webp')

im = im.resize((160,90)).convert("RGB")

img_array = np.array(im) / 255.0           # normalize if model trained on [0,1]
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

# --- Run prediction ---
pred = new_model.predict(img_array)

print("Raw prediction:", pred)